## Environtment Setup

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import string

pd.options.mode.chained_assignment = None

# from nltk.tokenize import word_tokenize
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from googletrans import Translator
pd.set_option('display.max_colwidth', None)

In [2]:
# nltk.download('punkt')

## Vader Setup

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
translator = Translator()

## Dataset Setup

In [4]:
df = pd.read_csv('../Dataset/tweet_scrape_result_merged.csv')
df.head()

,Datetime,Tweet Id,Tweet,Username,Language
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2299 entries, 0 to 2298
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Datetime  2299 non-null   object
 1   Tweet Id  2299 non-null   int64 
 2   Tweet     2299 non-null   object
 3   Username  2299 non-null   object
 4   Language  2299 non-null   object
dtypes: int64(1), object(4)
memory usage: 89.9+ KB


## Function Cleaning, Casefolding, Tokenizing, and Stemming

In [6]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', str(text)) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', str(text)) # remove hashtag
    text = re.sub(r'RT[\s]', '', str(text)) # remove RT
    text = re.sub(r"http\S+", '', str(text)) # remove link
    text = re.sub(r"https\S+", '', str(text)) # remove link
    text = re.sub(r'[0-9]+', '', str(text)) # remove numbers
    text = re.sub(r'(.)\1+', r'\1', str(text))


    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text

    return text

def casefoldingText(text): 
    text = text.lower() 
    return text

def tokenizingText(text): 
    text = word_tokenize(text) 
    return text

def acr(text):
    acr_dict = {
        'ri': 'indonesia',
        'bi': 'bank indonesia',
        'bbm' : 'bahan bakar minyak',
        'umkm' : 'usaha mikro kecil menengah',
        'menparekraf' : 'menteri pariwisita dan ekonomi kreatif',
        'cryto' : 'crypto',
        'menkeu' : 'menteri keuangan',
        'serem' : 'seram',
        'ojk' : 'otoritas jasa keungan',
        'gw' : 'saya',
        'bgt' : 'sangat',
        'mang' : 'memang',
        'cth' : 'contoh',
        'kominfo' : 'kementrian komunikasi dan informatika republik indonesia',
        'bumd' : 'badan usaha milik daearah',
        'bumn' : 'badan usaha milik negara'
    }

    text_words = text.split()
    for i, word in enumerate(text_words):
        if word.lower() in acr_dict:
            text_words[i] = acr_dict[word]
    return ' '.join(text_words)

def filteringText(text): # Remove stopwors in a text
    set_stop = stopwords.words('indonesian')
    set_stop.extend(["yg","dg","rt","dgn","ny","d",'klo', 'klu', 'adaekonomi', 'ad', 'adain', 'adb', 'al', 'ampdipuji', 'an', 'anieshrsfpi', 'arsjad', 'asf', 'askrl', 'bmw', 'blok', 'bni', 'bp', 'bpa', 
                     'bpjs', 'bri', 'brimo', 'btpn', 'buahahahahahahahahahahahahahahahahahahahahahahahahahahaha', 'betmen', 'bgd', 'bgmana', 'bgmn', 'bgnya', 
                     'awowkwowkwok', 'bagimn', 'anies', 'baswedan', 'aamin', 'abamg', 'abcde', 'abar', 'absorber', 'acep', 'leslar', 'hantu', 'gagak', 'anjir', 'erick', 'thohir', 'ceo', 'jpmorgan' ,
                     'chase', 'jamie', 'dimon', 'tv', 'yaa', 'allah', 'yesus', 'mmc', 'kaffah', 'svt', 'gais', 'skz','amp', 'belon', 'kalo', 'biar', 'lg', 'bikin', 'sri', 'anjing', 'layof', 
                     'jokowi', 'luhut', 'mulyani', 'herry', 'niagahoster', 'by', 'thn', 'kanker','bilang','gak', 'ga', 'krn', 'nya', 'nih', 'sih','si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'jd', 'jgn', 'sdh', 'aja', 'n', 'djp', 'sunarya', 't','nyg', 
                     'hehe', 'pen', 'u', 'nan', 'loh','rt','&amp', 'yah', 'byk', 'byr', 'cbi', 'cc', 'ce', 'cebong', 'diy', 'dki', 'dkt', 'dm', 'drs', 'dr', 'prof', 'elon', 'musk', 'yth', 'zero', 'zom', 'zh', 'ysl', 'yoy', 'yra', 'yorobun', 'ygy', 'yg', 'yl', 'yo'])
    listStopwords = set(set_stop)
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered 
    return text

def stemmingText(text): 
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): 
    sentence = ' '.join(word for word in list_words)
    return sentence

## Do Preprocessing

In [7]:
df['Tweet_clean'] = df['Tweet'].apply(cleaningText)
df['Tweet_clean'] = df['Tweet_clean'].apply(casefoldingText)

df.head()

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari


In [8]:
df['Tweet_clean'] = df['Tweet_clean'].apply(acr)

df.head()

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari


In [9]:
df['Tweet_preprocessed'] = df['Tweet_clean'].apply(tokenizingText)
df.head()

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean,Tweet_preprocessed
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di,"[indonesia, resesi, pengusaha, sebut, pertumbuhan, ekonomi, baru, bangkit, di]"
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[pengamat, mengatakan, pertumbuhan, ekonomi, baru, bisa, normal, pada, namun, jika, tidak, ada, perbaikan, dalam, penanganan, pandemi, dan, resesi, ekonomi, terus, berlanjut, hinga, indonesia, kemungkinan, akan, masuk, ke, jurang, depresi, ekonomi]"
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[pengamat, mengatakan, pertumbuhan, ekonomi, baru, bisa, normal, pada, namun, jika, tidak, ada, perbaikan, dalam, penanganan, pandemi, dan, resesi, ekonomi, terus, berlanjut, hinga, indonesia, kemungkinan, akan, masuk, ke, jurang, depresi, ekonomi]"
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d,"[kalau, kanker, sepertinya, sudah, masuk, stadium, empat, sudah, keropos, menungu, ambruk, bagai, anjing, mencret, yg, sudah, kehilangan, tenaga, ini, bukan, kabar, baik, ekonomi, indonesia, diprediksi, baru, sembuh, dari, resesi, pada, kebangkrutan, masal, sulit, d]"
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari,"[ini, bukan, kabar, baik, ekonomi, indonesia, diprediksi, baru, sembuh, dari, resesi, pada, kebangkrutan, masal, sulit, dihindari]"


In [10]:
df['Tweet_preprocessed'] = df['Tweet_preprocessed'].apply(filteringText)
df.head()

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean,Tweet_preprocessed
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di,"[indonesia, resesi, pengusaha, pertumbuhan, ekonomi, bangkit]"
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[pengamat, pertumbuhan, ekonomi, normal, perbaikan, penanganan, pandemi, resesi, ekonomi, berlanjut, hinga, indonesia, masuk, jurang, depresi, ekonomi]"
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[pengamat, pertumbuhan, ekonomi, normal, perbaikan, penanganan, pandemi, resesi, ekonomi, berlanjut, hinga, indonesia, masuk, jurang, depresi, ekonomi]"
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d,"[masuk, stadium, keropos, menungu, ambruk, mencret, kehilangan, tenaga, kabar, ekonomi, indonesia, diprediksi, sembuh, resesi, kebangkrutan, masal, sulit]"
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari,"[kabar, ekonomi, indonesia, diprediksi, sembuh, resesi, kebangkrutan, masal, sulit, dihindari]"


In [11]:
df['Tweet_preprocessed'] = df['Tweet_preprocessed'].apply(stemmingText)
df.head()

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean,Tweet_preprocessed
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di,"[indonesia, resesi, usaha, tumbuh, ekonomi, bangkit]"
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[amat, tumbuh, ekonomi, normal, baik, tangan, pandemi, resesi, ekonomi, lanjut, hinga, indonesia, masuk, jurang, depresi, ekonomi]"
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[amat, tumbuh, ekonomi, normal, baik, tangan, pandemi, resesi, ekonomi, lanjut, hinga, indonesia, masuk, jurang, depresi, ekonomi]"
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d,"[masuk, stadium, keropos, tungu, ambruk, mencret, hilang, tenaga, kabar, ekonomi, indonesia, prediksi, sembuh, resesi, bangkrut, masal, sulit]"
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari,"[kabar, ekonomi, indonesia, prediksi, sembuh, resesi, bangkrut, masal, sulit, hindar]"


In [12]:
df['Clean_Sentence'] = df['Tweet_preprocessed'].apply(toSentence)

df.head()

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean,Tweet_preprocessed,Clean_Sentence
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di,"[indonesia, resesi, usaha, tumbuh, ekonomi, bangkit]",indonesia resesi usaha tumbuh ekonomi bangkit
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[amat, tumbuh, ekonomi, normal, baik, tangan, pandemi, resesi, ekonomi, lanjut, hinga, indonesia, masuk, jurang, depresi, ekonomi]",amat tumbuh ekonomi normal baik tangan pandemi resesi ekonomi lanjut hinga indonesia masuk jurang depresi ekonomi
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[amat, tumbuh, ekonomi, normal, baik, tangan, pandemi, resesi, ekonomi, lanjut, hinga, indonesia, masuk, jurang, depresi, ekonomi]",amat tumbuh ekonomi normal baik tangan pandemi resesi ekonomi lanjut hinga indonesia masuk jurang depresi ekonomi
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d,"[masuk, stadium, keropos, tungu, ambruk, mencret, hilang, tenaga, kabar, ekonomi, indonesia, prediksi, sembuh, resesi, bangkrut, masal, sulit]",masuk stadium keropos tungu ambruk mencret hilang tenaga kabar ekonomi indonesia prediksi sembuh resesi bangkrut masal sulit
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari,"[kabar, ekonomi, indonesia, prediksi, sembuh, resesi, bangkrut, masal, sulit, hindar]",kabar ekonomi indonesia prediksi sembuh resesi bangkrut masal sulit hindar


## Removing Duplicate & Export

In [13]:
df.duplicated(subset='Clean_Sentence', keep='first').sum()  # count duplicated

373

In [14]:
df.drop_duplicates(subset='Clean_Sentence', inplace=True) # remove duplicated

In [15]:
print(df.info(), df.duplicated(subset='Clean_Sentence', keep='first').sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1926 entries, 0 to 2298
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Datetime            1926 non-null   object
 1   Tweet Id            1926 non-null   int64 
 2   Tweet               1926 non-null   object
 3   Username            1926 non-null   object
 4   Language            1926 non-null   object
 5   Tweet_clean         1926 non-null   object
 6   Tweet_preprocessed  1926 non-null   object
 7   Clean_Sentence      1926 non-null   object
dtypes: int64(1), object(7)
memory usage: 135.4+ KB
None 0


In [47]:
df.to_csv(r'Clean_Dataset.csv', index = False, header = True,index_label=None)

In [48]:
import numpy as np 
import pandas as pd 

from googletrans import Translator
import pandas as pd

In [49]:
df1 = pd.read_csv('Clean_Dataset.csv')

In [50]:
from googletrans import Translator
import pandas as pd

translator = Translator()

df1['English'] = df1['Clean_Sentence'].apply(translator.translate, src='id', dest='en')
df1['English'] = df1['English'].apply(getattr, args=('text',))
df1.head()

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean,Tweet_preprocessed,Clean_Sentence,English
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di,"['indonesia', 'resesi', 'usaha', 'tumbuh', 'ekonomi', 'bangkit']",indonesia resesi usaha tumbuh ekonomi bangkit,indonesia recession business grows economy rises
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"['amat', 'tumbuh', 'ekonomi', 'normal', 'baik', 'tangan', 'pandemi', 'resesi', 'ekonomi', 'lanjut', 'hinga', 'indonesia', 'masuk', 'jurang', 'depresi', 'ekonomi']",amat tumbuh ekonomi normal baik tangan pandemi resesi ekonomi lanjut hinga indonesia masuk jurang depresi ekonomi,very normal economic growth both hands of the pandemic continued economic recession until Indonesia entered the abyss of economic depression
2,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d,"['masuk', 'stadium', 'keropos', 'tungu', 'ambruk', 'mencret', 'hilang', 'tenaga', 'kabar', 'ekonomi', 'indonesia', 'prediksi', 'sembuh', 'resesi', 'bangkrut', 'masal', 'sulit']",masuk stadium keropos tungu ambruk mencret hilang tenaga kabar ekonomi indonesia prediksi sembuh resesi bangkrut masal sulit,"Entering the porous stage, waiting to collapse, diarrhea, loss of energy, Indonesian economic news predicts recovery from recession, mass bankruptcy, difficult"
3,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari,"['kabar', 'ekonomi', 'indonesia', 'prediksi', 'sembuh', 'resesi', 'bangkrut', 'masal', 'sulit', 'hindar']",kabar ekonomi indonesia prediksi sembuh resesi bangkrut masal sulit hindar,"Indonesian economic news predicts recovery from recession, mass bankruptcy, difficult to avoid"
4,2020-09-23 10:03:15+00:00,1308708497643597824,"Resesi Tak Terhindarkan, Ekonomi Diproyeksi Pulih Total pada 2023 #ddtcnews https://t.co/K7Rhi2rEZf",agg90,in,resesi tak terhindarkan ekonomi diproyeksi pulih total pada,"['resesi', 'hindar', 'ekonomi', 'proyeksi', 'pulih', 'total']",resesi hindar ekonomi proyeksi pulih total,"avoid a recession, the projected economy will recover completely"


In [51]:
df1.head()

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean,Tweet_preprocessed,Clean_Sentence,English
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di,"['indonesia', 'resesi', 'usaha', 'tumbuh', 'ekonomi', 'bangkit']",indonesia resesi usaha tumbuh ekonomi bangkit,indonesia recession business grows economy rises
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"['amat', 'tumbuh', 'ekonomi', 'normal', 'baik', 'tangan', 'pandemi', 'resesi', 'ekonomi', 'lanjut', 'hinga', 'indonesia', 'masuk', 'jurang', 'depresi', 'ekonomi']",amat tumbuh ekonomi normal baik tangan pandemi resesi ekonomi lanjut hinga indonesia masuk jurang depresi ekonomi,very normal economic growth both hands of the pandemic continued economic recession until Indonesia entered the abyss of economic depression
2,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d,"['masuk', 'stadium', 'keropos', 'tungu', 'ambruk', 'mencret', 'hilang', 'tenaga', 'kabar', 'ekonomi', 'indonesia', 'prediksi', 'sembuh', 'resesi', 'bangkrut', 'masal', 'sulit']",masuk stadium keropos tungu ambruk mencret hilang tenaga kabar ekonomi indonesia prediksi sembuh resesi bangkrut masal sulit,"Entering the porous stage, waiting to collapse, diarrhea, loss of energy, Indonesian economic news predicts recovery from recession, mass bankruptcy, difficult"
3,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari,"['kabar', 'ekonomi', 'indonesia', 'prediksi', 'sembuh', 'resesi', 'bangkrut', 'masal', 'sulit', 'hindar']",kabar ekonomi indonesia prediksi sembuh resesi bangkrut masal sulit hindar,"Indonesian economic news predicts recovery from recession, mass bankruptcy, difficult to avoid"
4,2020-09-23 10:03:15+00:00,1308708497643597824,"Resesi Tak Terhindarkan, Ekonomi Diproyeksi Pulih Total pada 2023 #ddtcnews https://t.co/K7Rhi2rEZf",agg90,in,resesi tak terhindarkan ekonomi diproyeksi pulih total pada,"['resesi', 'hindar', 'ekonomi', 'proyeksi', 'pulih', 'total']",resesi hindar ekonomi proyeksi pulih total,"avoid a recession, the projected economy will recover completely"


Penilaian compound atau skor total ini mengikuti jurnal terdahalu yang juga sesuai dengan standar dari Vader Sentiment seperti pada gambar berikut 

https://github.com/cjhutto/vaderSentiment

In [52]:

scores=[]
composes=[]
for i in range(len(df1['English'])):
    
    score = analyser.polarity_scores(df1['English'][i])
    compose = score
    score=score['compound']
    scores.append(score)
    composes.append(compose)
sentiment=[]
for i in scores:
    if i>=0.05: # nilai compound lebih dari sama ddengan posifit
        sentiment.append('Positive')
    elif i<=(-0.05): # nilai compound kurang dari sama ddengan negatif
        sentiment.append('Negative')
    else: # nilai compound diantara -0.05 - +0.05 sama ddengan netral
        sentiment.append('Neutral')
df1['Sentiment']=pd.Series(np.array(sentiment))
df1['Composes']=pd.Series(np.array(composes))
# df1.head()

In [55]:
df1['Sentiment'].value_counts()

Negative    1437
Positive     382
Neutral      107
Name: Sentiment, dtype: int64

In [56]:
print(scores)

[-0.4215, -0.5627, -0.8271, -0.7579, -0.6124, 0.0258, -0.6808, -0.4215, -0.4215, 0.7269, 0.1779, -0.8176, -0.5267, 0.128, -0.4215, -0.6597, -0.6705, -0.4215, -0.4215, 0.4588, -0.7964, 0.4215, -0.0772, -0.4215, -0.4215, -0.3612, -0.4588, -0.4404, -0.4215, -0.4215, 0.5423, 0.128, -0.4215, -0.7845, 0.34, 0.4019, 0.4588, -0.4215, -0.4215, -0.6249, 0.1779, -0.4215, 0.296, -0.6249, -0.3612, -0.296, 0.7003, 0.1779, -0.4215, -0.2732, -0.7351, 0.7845, -0.5859, -0.3612, -0.3182, -0.7003, -0.4215, -0.2644, -0.7003, -0.4215, -0.128, -0.4215, 0.6486, 0.4404, -0.4215, -0.0772, -0.4215, 0.1154, 0.7269, 0.4865, -0.7506, 0.1513, -0.8689, -0.4215, -0.4215, -0.7964, -0.4215, -0.4215, -0.1531, 0.0, -0.4215, 0.1779, -0.4215, 0.7269, -0.6808, -0.2732, -0.7351, -0.0258, -0.5574, -0.34, 0.2732, -0.1531, 0.1027, 0.3182, 0.4767, -0.4939, -0.5994, -0.4215, -0.7783, -0.7269, 0.4767, 0.4215, 0.8176, 0.4215, -0.1531, -0.4215, 0.8442, -0.7351, -0.6705, 0.5859, -0.2263, -0.4215, 0.2263, 0.296, -0.9042, 0.6597, -0.421

In [57]:
df1.to_csv(r'Clean_With_Vader.csv', index = False, header = True,index_label=None)

## Sentimen Score Base On Jurnal and Score Vader

In [59]:
df2 = pd.read_csv('Clean_With_Vader.csv')
# df2 = df2.drop(columns=['Datetime', 'Tweet Id', 'Username', 'Language'])

In [60]:
df_negatif = df2.loc[df2['Sentiment'] == 'Negative']
df_negatif.head(10)

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean,Tweet_preprocessed,Clean_Sentence,English,Sentiment,Composes
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di,"['indonesia', 'resesi', 'usaha', 'tumbuh', 'ekonomi', 'bangkit']",indonesia resesi usaha tumbuh ekonomi bangkit,indonesia recession business grows economy rises,Negative,"{'neg': 0.359, 'neu': 0.641, 'pos': 0.0, 'compound': -0.4215}"
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"['amat', 'tumbuh', 'ekonomi', 'normal', 'baik', 'tangan', 'pandemi', 'resesi', 'ekonomi', 'lanjut', 'hinga', 'indonesia', 'masuk', 'jurang', 'depresi', 'ekonomi']",amat tumbuh ekonomi normal baik tangan pandemi resesi ekonomi lanjut hinga indonesia masuk jurang depresi ekonomi,very normal economic growth both hands of the pandemic continued economic recession until Indonesia entered the abyss of economic depression,Negative,"{'neg': 0.247, 'neu': 0.645, 'pos': 0.109, 'compound': -0.5627}"
2,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d,"['masuk', 'stadium', 'keropos', 'tungu', 'ambruk', 'mencret', 'hilang', 'tenaga', 'kabar', 'ekonomi', 'indonesia', 'prediksi', 'sembuh', 'resesi', 'bangkrut', 'masal', 'sulit']",masuk stadium keropos tungu ambruk mencret hilang tenaga kabar ekonomi indonesia prediksi sembuh resesi bangkrut masal sulit,"Entering the porous stage, waiting to collapse, diarrhea, loss of energy, Indonesian economic news predicts recovery from recession, mass bankruptcy, difficult",Negative,"{'neg': 0.374, 'neu': 0.554, 'pos': 0.073, 'compound': -0.8271}"
3,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari,"['kabar', 'ekonomi', 'indonesia', 'prediksi', 'sembuh', 'resesi', 'bangkrut', 'masal', 'sulit', 'hindar']",kabar ekonomi indonesia prediksi sembuh resesi bangkrut masal sulit hindar,"Indonesian economic news predicts recovery from recession, mass bankruptcy, difficult to avoid",Negative,"{'neg': 0.455, 'neu': 0.545, 'pos': 0.0, 'compound': -0.7579}"
4,2020-09-23 10:03:15+00:00,1308708497643597824,"Resesi Tak Terhindarkan, Ekonomi Diproyeksi Pulih Total pada 2023 #ddtcnews https://t.co/K7Rhi2rEZf",agg90,in,resesi tak terhindarkan ekonomi diproyeksi pulih total pada,"['resesi', 'hindar', 'ekonomi', 'proyeksi', 'pulih', 'total']",resesi hindar ekonomi proyeksi pulih total,"avoid a recession, the projected economy will recover completely",Negative,"{'neg': 0.417, 'neu': 0.583, 'pos': 0.0, 'compound': -0.6124}"
6,2020-07-16 04:16:41+00:00,1283616518257836032,@CNNIndonesia Negara Singapura kena resesi ekonomi m

In [61]:
df_negatif = df2.loc[df2['Sentiment'] == 'Positive']
df_negatif.head(10)

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean,Tweet_preprocessed,Clean_Sentence,English,Sentiment,Composes
9,2022-12-30 16:05:03+00:00,1608856700882407424,KETAHANAN pangan harus menjadi resolusi 2023 seluruh pemangku kepentingan. Pemerintah dan swasta harus lebih kuat lagi menjadi ketahanan pangan mengingat tahun depan diprediksi bakal terjadi resesi ekonomi. https://t.co/loiwcPVJYt,mediaindonesia,in,ketahanan pangan harus menjadi resolusi seluruh pemangku kepentingan pemerintah dan swasta harus lebih kuat lagi menjadi ketahanan pangan mengingat tahun depan diprediksi bakal terjadi resesi ekonomi,"['tahan', 'pangan', 'resolusi', 'mang', 'penting', 'perintah', 'swasta', 'kuat', 'tahan', 'pangan', 'prediksi', 'resesi', 'ekonomi']",tahan pangan resolusi mang penting perintah swasta kuat tahan pangan prediksi resesi ekonomi,resolution of food security is indeed important to command the private sector to be strong in food security in predicting an economic recession,Positive,"{'neg': 0.091, 'neu': 0.586, 'pos': 0.322, 'compound': 0.7269}"
10,2022-12-30 14:36:04+00:00,1608834307761786887,NARASI RESESI VS OPTIMISME EKONOMI 2023 https://t.co/PbCz681zXE via @Nawacita,nawacitadotco,in,narasi resesi vs optimisme ekonomi via,"['narasi', 'resesi', 'vs', 'optimisme', 'ekonomi', 'via']",narasi resesi vs optimisme ekonomi via,narrative of recession vs economic optimism via,Positive,"{'neg': 0.248, 'neu': 0.442, 'pos': 0.31, 'compound': 0.1779}"
13,2022-12-30 10:42:06+00:00,1608775427744763905,Presiden Jokowi pastikan kondisi ekonomi RI stabil dan siap hadapi resesi 2023 #JokowiPresidenku #PembangunanUntukRakyat #PresidenJokowiHebat #JokowiMembangunNegeri #JokowiMajuBersamaIndonesia #JokowiBapakInfrastruktur https://t.co/TiB9IkWszg,DipaDuaLima,in,presiden jokowi pastikan kondisi ekonomi indonesia stabil dan siap hadapi resesi,"['presiden', 'pasti', 'kondisi', 'ekonomi', 'indonesia', 'stabil', 'hadap', 'resesi']",presiden pasti kondisi ekonomi indonesia stabil hadap resesi,the president is certain that Indonesia's economic condition is stable in the face of a recession,Positive,"{'neg': 0.139, 'neu': 0.647, 'pos': 0.214, 'compound': 0.128}"
19,2022-12-30 06:35:19+00:00,1608713322697834498,Mantap.. Presiden Jokowi pastikan kondisi ekonomi RI stabil dan siap hadapi resesi 2023 #JokowiPresidenku #PembangunanUntukRakyat #PresidenJokowiHebat #JokowiMembangunNegeri #JokowiMajuBersamaIndonesia #JokowiBapakInfrastruktur https://t.co/zO3TOkutQa,deniangria,in,mantap presiden jokowi pastikan kondisi ekonomi indonesia stabil dan siap hadapi resesi,"['mantap', 'presiden', 'pasti', 'kondisi', 'ekonomi', 'indonesia', 'stabil', 'hadap', 'resesi']",mantap presiden pasti kondisi ekonomi indonesia stabil hadap resesi,sure the president is sure that Indonesia's economic condition is stable in the face of a recession,Positive,"{'neg': 0.124, 'neu': 0.575, 'pos': 0.301, 'compound': 0.4588}"
21,2022-12-30 02:32:02+00:00,1608652098815811586,"Apindo Ingatkan Pemerintah Waspadai Resesi Tahun 2023: Tantangan ekonomi di fase perfect storm tahun depan nanti akan dihadapi oleh sejumlah negara, termasuk Indonesia. Pemerintah diminta untuk mewaspadai terjadinya resesi di tahun depan.\n\nbaca… https://t.co/mQQ5IYhQKG",RMOLJatim_id,in,apindo ingatkan pemerintah waspadai resesi tahun tantangan ekonomi di fase perfect storm tahun depan nanti akan dihadapi oleh sejumlah negara termasuk indonesia pemerintah diminta untuk mewaspadai terjadinya resesi di tahun depan baca…,"['apindo', 'ingat', 'perintah', 'waspada', 'resesi', 'tantang', 'ekonomi', 'fase', 'perfect', 'storm', 'hadap', 'negara', 'indonesia', 'perintah', 'waspada', 'resesi', 'baca']",apindo ingat perintah waspada resesi tantang ekonomi fase perfect storm hadap negara indonesia perintah waspada resesi baca,"Apindo remembers the recession alert order, challenge the perfect storm phase of the economy towards the Indonesian state, read the recession alert order",Positive,"{'neg': 0.175, 'neu': 0.531, 'pos': 0.294, '

In [62]:
df_negatif = df2.loc[df2['Sentiment'] == 'Neutral']
df_negatif.head(10)

,Datetime,Tweet Id,Tweet,Username,Language,Tweet_clean,Tweet_preprocessed,Clean_Sentence,English,Sentiment,Composes
5,2020-08-17 04:23:26+00:00,1295214630051000326,Semoga Indonesia dpt berhasil ujicoba vaksin covid-19 di atas 90% shg tdk terkena resesi krisis yg diprediksikan sd 2023... tetap optimis hadapi terbenamnya ekonomi.,ankrizz,in,semoga indonesia dpt berhasil ujicoba vaksin covid di atas shg tdk terkena resesi krisis yg diprediksikan sd tetap optimis hadapi terbenamnya ekonomi,"['moga', 'indonesia', 'dpt', 'hasil', 'ujicoba', 'vaksin', 'covid', 'shg', 'kena', 'resesi', 'krisis', 'prediksi', 'sd', 'optimis', 'hadap', 'benam', 'ekonomi']",moga indonesia dpt hasil ujicoba vaksin covid shg kena resesi krisis prediksi sd optimis hadap benam ekonomi,"I hope that Indonesia will get the results of the Covid vaccine trial, which is hit by a recession",Neutral,"{'neg': 0.123, 'neu': 0.749, 'pos': 0.128, 'compound': 0.0258}"
79,2022-12-26 05:50:41+00:00,1607252541494165504,"Tahun 2022 akan berakhir beberapa hari lagi. Jika kamu dapat kembali ke masa lalu, apa yang akan kamu katakan pada dirimu yang lebih muda?\n\n#2023goals #2023ready #2022year #resesi #resesi2023 #ResesiDunia #resesiglobal #resesiekonomi #resesiindonesia #resesiekonomi2023 https://t.co/S0mgpGHXcR",rizalconsulting,in,tahun akan berakhir beberapa hari lagi jika kamu dapat kembali ke masa lalu apa yang akan kamu katakan pada dirimu yang lebih muda,"['diri', 'muda']",diri muda,young self,Neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
87,2022-12-25 14:40:33+00:00,1607023497360080896,@Hilmi28 Semoga th 2023 secara ekonomi keluarga lebih baik dari th 2022...walaupun dikatakan 2023 dimulainya Resesi Global Aamiin YRA,BudiSul27538925,in,semoga th secara ekonomi keluarga lebih baik dari th walaupun dikatakan dimulainya resesi global aamin yra,"['moga', 'th', 'ekonomi', 'keluarga', 'th', 'resesi', 'global']",moga th ekonomi keluarga th resesi global,hopefully th family economy th global recession,Neutral,"{'neg': 0.267, 'neu': 0.476, 'pos': 0.257, 'compound': -0.0258}"
117,2022-12-21 13:50:03+00:00,1605561235998912512,"Hadapi Resesi 2023, Ini Usaha IRT Modal Dikit Untung Banyak #LengkapCepatBeritanya #Keuangan #Ekonomi #Ekonomi #EkonomiIndonesia . https://t.co/ZlwcjzCI8T",okezonenews,in,hadapi resesi ini usaha imodal dikit untung banyak,"['hadap', 'resesi', 'usaha', 'imodal', 'dikit', 'untung']",hadap resesi usaha imodal dikit untung,"Facing a recession, capital business has a little profit",Neutral,"{'neg': 0.226, 'neu': 0.564, 'pos': 0.21, 'compound': -0.0498}"
148,2022-12-19 08:35:42+00:00,1604757351499567105,"Kerja sama internasional yang dilakukan Indonesia, diantaranya seperti dengan Australia, mempunyai tujuan untuk memajukan atau memulihkan ekonomi yang diprediksi mengalami resesi pada tahun 2023. https://t.co/zlJcDqu0hl",JayaKatwang15,in,kerja sama internasional yang dilakukan indonesia diantaranya seperti dengan australia mempunyai tujuan untuk memajukan atau memulihkan ekonomi yang diprediksi mengalami resesi pada tahun,"['kerja', 'internasional', 'indonesia', 'australia', 'tuju', 'maju', 'pulih', 'ekonomi', 'prediksi', 'alami', 'resesi']",kerja internasional indonesia australia tuju maju pulih ekonomi prediksi alami resesi,international work Indonesia Australia aims to progress to recover the economy is predicted to experience a recession,Neutral,"{'neg': 0.136, 'neu': 0.728, 'pos': 0.136, 'compound': 0.0}"
155,2022-12-19 02:38:41+00:00,1604667507339448322,3. Kondisi ini memberikan optimisme bagi Indonesia dalam pencapaian target pertumbuhan ekonomi nasional 2022 di tengah perlambatan ekonomi global serta modal dasar menghadapi ancaman resesi ekonomi global 2023.\n\n\n#GenjotEkspor \n#AyoDagang,Kemendag,in,kondisi ini memberikan optimisme bagi indonesia dalam pencapaian target pertumbuhan ekonomi nasional di tengah perlambatan ekonomi global serta modal dasar menghadapi ancaman resesi ekonomi global,"['kondisi', 'optimisme', 'indonesia', 'capa